##  k Nearest-Neighbors

In pattern recognition, the k-nearest neighbors algorithm (k-NN) is a non-parametric method used for classification and regression. In both cases, the input consists of the k closest training examples in the feature space. The output depends on whether k-NN is used for classification or regression:

1. In k-NN classification, the output is a class membership. An object is classified by a **plurality vote of its neighbors, with the object being assigned to the class most common among its k nearest neighbors** (k is a positive integer, typically small). If k = 1, then the object is simply assigned to the class of that single nearest neighbor.

2. In k-NN regression, the output is the property value for the object. This value is the **average of the values of its k nearest neighbors**.

The k-NN algorithm is among the simplest of all machine learning algorithms. 

In [1]:
#CSV (Comma Separated Values) format is the most common import and export format for spreadsheets and databases.
import csv
import random
import math
#The operator module exports a set of efficient functions corresponding to the intrinsic operators of Python.
import operator

## Example

In [2]:
A = tuple((np.array([1,2,3]),'blue'))   
B = tuple((np.array([1,2,3]),'orange')) 
C = tuple((np.array([1,2,3]),'orange')) 
Total = np.hstack((A,B))
Total = np.hstack((Total,C))

print ('Total',Total)
print ('Total',Total[3])

classVotes = {}
#print ('len(neighbors)',len(neighbors))
for x in range(0,5,2):
    response = Total[x+1]
    print('x',x,'response',response)
    response = str(response)
    if response in classVotes:
        classVotes[response] += 1
    else:
        classVotes[response] = 1
        
print ('classVotes.items():',classVotes.items())        
sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True) 
print('-------------------------')
print(sortedVotes)
print(sortedVotes[0][0])

Total [array([1, 2, 3]) 'blue' array([1, 2, 3]) 'orange' array([1, 2, 3])
 'orange']
Total orange
x 0 response blue
x 2 response orange
x 4 response orange
classVotes.items(): dict_items([('orange', 2), ('blue', 1)])
-------------------------
[('orange', 2), ('blue', 1)]
orange


## k Nearest-Neighbors implementation

In [3]:
# Split the data into training and test data
def loadDataset(filename, split, trainingSet=[] , testSet=[]):
    with open(filename, 'r') as csvfile:
        #rb: 'r' for reading, 'w' for writing, opening a binary file 'b'
        lines = csv.reader(csvfile)
        #print('lines',type(lines))
        dataset = list(lines)
        #print('dataset',type(dataset),'len(dataset)',len(dataset))
        le = len(dataset)
        for x in range(len(dataset)):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
#Return the next random floating point number in the range [0.0, 1.0)
# According to the split constant the dataset is divided.
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])

    return le
                              

The Euclidean distance or Euclidean metric is the "ordinary" straight-line distance between two points in Euclidean space.

$$\begin{aligned} d ( \mathbf { p } , \mathbf { q } ) = d ( \mathbf { q } , \mathbf { p } ) & = \sqrt { \left( q _ { 1 } - p _ { 1 } \right) ^ { 2 } + \left( q _ { 2 } - p _ { 2 } \right) ^ { 2 } + \cdots + \left( q _ { n } - p _ { n } \right) ^ { 2 } } \\ & = \sqrt { \sum _ { i = 1 } ^ { n } \left( q _ { i } - p _ { i } \right) ^ { 2 } } \end{aligned}$$

In [4]:
def euclideanDistance(instance1, instance2, length):
    distance = 0
    # For each sample the euclideanDistance is found takin in to account the four features. 
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

In [5]:
def getNeighbors(trainingSet, testInstance, k):
    distances = []
#The value of -1 is important. It is beacuse I am evaluating a point, hence, the lengh of the datset is N-1
    length = len(testInstance)-1
    #print ('length',length)
    for x in range(len(trainingSet)):
        #print ('x',x)
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    #The sort() method sorts the elements of a given list in a specific order. 
    #operator.itemgetter(1) the column distance.
    distances.sort(key = operator.itemgetter(1))
    neighbors = []
    
    #I take the 3 closest neighbors    
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

<img src="images/vote.png" style="width: 250px;">

In [6]:
def getResponse(neighbors):
    # Creating a list with all the possible neighbors
    classVotes = {}
    #print ('len(neighbors)',len(neighbors))
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    # sortedVotes choose the label according ti its neigboorhs.
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

In [7]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [8]:
def main():
    trainingSet=[]
    testSet=[]
    split = 0.67
    
    a = loadDataset('/home/johan/repos/GitHub/Introduction-to-Machine-Learning/Datasets/iris.data', split, trainingSet, testSet)
    print ('Train set: ',(len(trainingSet)),' Percentage:', ((len(trainingSet))/a)*100,'%')
    print ('Test set: ' , (len(testSet)),'  Percentage:',((len(testSet))/a)*100,'%')    
    predictions=[]
    k = 3
    
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        
    accuracy = getAccuracy(testSet, predictions)
    print ('Accuracy: ', accuracy)

Usually it is better to normalize attributes before for calculating distance. The reason is there can be attributes which are in large range initially and they can outweighing the attributes with initially smaller ranges

In [9]:
main()

Train set:  104  Percentage: 69.33333333333334 %
Test set:  46   Percentage: 30.666666666666664 %
Accuracy:  97.82608695652173
